In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta

def load_data():
    train = pd.read_csv('C:\kaggle_data/credit/train.csv')
    test = pd.read_csv('C:\kaggle_data/credit/test.csv')
    ss = pd.read_csv('C:\kaggle_data/credit/submission.csv')
    return train, test, ss


def resam_1w(df):
    df = df.drop(['card_id','time','days_of_week','holyday','installments'],1)
    df['date_idx'] = pd.DatetimeIndex(df['date'])
    df = df.set_index('date_idx')
    store_lst = []
    for store_id in df['store_id'].unique():
    
        temp = df[df['store_id']==store_id]
        temp_0 = temp['amount'].resample('1W').sum()
        temp_0 = pd.DataFrame(temp_0)
        temp_0['target'] = temp_0['amount'].shift(-1).fillna(0).astype('int')
        temp_0 = temp_0.reset_index()
        
        temp_0['year'], temp_0['month'], temp_0['day'] = 0,0,0
        for i in range(len(temp_0)):
            temp_0.loc[i,'year'] = temp_0['date_idx'][i].year
            temp_0.loc[i,'month'] = temp_0['date_idx'][i].month
            temp_0.loc[i, 'day'] = temp_0['date_idx'][i].day
        temp_0['store_id'] = store_id
        temp_0['counts'] = len(temp_0)
        store_lst.append(temp_0)
        
    return pd.concat(store_lst)

In [4]:
%%time

train, test, ss = load_data()

train = resam_1w(train)
test = resam_1w(test)

train = train.drop('date_idx',1)
train['year'] = train['year'] - 2016
train['day'] = train['day']/31

y = train.pop('target')
train = train[['counts', 'year', 'month', 'day','amount' ]]

# test = test.drop('date_idx',1)
test['year'] = test['year'] - 2016
test['day'] = test['day']/31

import xgboost as xgb
# from sklearn.model_selection import KFold

xgb_reg = xgb.XGBRegressor(random_state=1123, tree_method='gpu_hist')

xgb_reg.fit(train, y)
for s_id in test['store_id'].unique():
    temp = test[test['store_id']==s_id]
    for i in range(14):
        last_row = temp.tail(1)
        counts = last_row['counts'].values[0]
        store_id = last_row['store_id'].values[0]
        next_date = (last_row['date_idx'] + timedelta(days=7)).iloc[-1]
        target = xgb_reg.predict(last_row[['counts', 'year', 'month', 'day', 'amount']])
        temp = temp.append({'date_idx':next_date, 
                     'amount':target[0], 
                     'year':next_date.year-2016,
                     'month':next_date.month,
                     'day':next_date.day/31,
                     'store_id': store_id,
                     'counts':counts}, ignore_index=True)
    
    
    ss.iloc[s_id][1] = temp.iloc[-14:]['amount'].sum()/98*100
    
ss.to_csv('half_baseline.csv',index=False)
print('됐습니당')

됐습니당
Wall time: 1min 53s
